In [20]:
from joblib import load
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from joblib import load, dump
import numpy as np
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from scikeras.wrappers import KerasClassifier
import optuna



In [21]:
combined_data = load('../../DataDumps/all_data_fft - balanced YesandNo.joblib')
combined_labels = load('../../DataDumps/2class_labels.joblib')
X_train_val, X_test, y_train_val, y_test = train_test_split(combined_data, combined_labels, test_size=0.2, random_state=42)

In [22]:
def create_model(lstm_units=100, dropout_rate=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(units=lstm_units, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=lstm_units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [26]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)  
scores = []
for train_index, test_index in skf.split(X_train_val, y_train_val):
    X_tr, X_val = X_train_val[train_index], X_train_val[test_index]
    y_tr, y_val = y_train_val[train_index], y_train_val[test_index]
    model = create_model()
    model.fit(X_tr, y_tr)
    scores.append(model.evaluate(X_val, y_val))


11/11 [==============================] - 0s 3ms/step - loss: 0.5000 - accuracy: 0.7462


In [4]:
model = create_model()

In [10]:
history = model.fit(X_train, y_train, epochs=150)

Epoch 1/150
52/52 [==============================] - 0s 5ms/step - loss: 0.3367 - accuracy: 0.8330
Epoch 2/150
52/52 [==============================] - 0s 5ms/step - loss: 0.3515 - accuracy: 0.8131
Epoch 3/150
52/52 [==============================] - 0s 5ms/step - loss: 0.3463 - accuracy: 0.8282
Epoch 4/150
52/52 [==============================] - 0s 5ms/step - loss: 0.3450 - accuracy: 0.8288
Epoch 5/150
52/52 [==============================] - 0s 5ms/step - loss: 0.3482 - accuracy: 0.8240
Epoch 6/150
52/52 [==============================] - 0s 5ms/step - loss: 0.3376 - accuracy: 0.8312
Epoch 7/150
52/52 [==============================] - 0s 5ms/step - loss: 0.3361 - accuracy: 0.8324
Epoch 8/150
52/52 [==============================] - 0s 5ms/step - loss: 0.3447 - accuracy: 0.8294
Epoch 9/150
52/52 [==============================] - 0s 5ms/step - loss: 0.3402 - accuracy: 0.8312
Epoch 10/150
52/52 [==============================] - 0s 5ms/step - loss: 0.3553 - accuracy: 0.8210
Epoch 11/

In [12]:
y_pred = model.predict(X_test)
y_pred = np.where(y_pred >= 0.5, 1, 0) 

test_acc = accuracy_score(y_test, y_pred)
print('Test Accuracy:', test_acc)

13/13 [==============================] - 0s 4ms/step
Test Accuracy: 0.7807228915662651
